מגישות:
ירדן אוחנה 
315321851
עמית ששון 
322759648

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import time
from IPython.display import HTML
from urllib.parse import urlparse
from datetime import datetime, timedelta
import calendar
import numpy as np

In [2]:
url = 'https://www.ad.co.il/car?sp261=19452'

In [3]:
response = requests.get(url)

In [4]:
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
base_url = 'https://www.ad.co.il'
car_ads_url = f'{base_url}/car?sp261=19452&pageindex='

data1 = []

# אינדקס דף התחלתי
page_index = 1

# טווח השנים להכללה לפי בקשת המרצה
#למרות שכרגע זה כולל את כל המודעות הוספנו כדי שהקוד יוכל לשמש גם בעתיד אם ירצו להריץ עוד כמה שנים
start_year = 1980
end_year = 2024

# לולאה כללית שרצה על כל העמודים עד שהיא מגיעה לדף ריק שאין בו נתונים
#בחרנו לולאה כללים שרצה על העמודים ולא לולאה לפי מספר העמודים שקיימים עכשיו כי כל הזמן מתווסף ויורד מודעות ועמודים משתנים
#ורצינו קוד כללי שירוץ תמיד
while True:
    page_url = car_ads_url + str(page_index)
    page_response = requests.get(page_url)
    
    if page_response.status_code == 200:
        page_soup = BeautifulSoup(page_response.content, 'html.parser')
        
        # מציאת כל הקישורים למודעות רכב
        all_links = page_soup.find_all('a', href=True)
        
        # יצירת רשימה של כל הקישורים המכילים '/ad/' והכותרת שמתחילה ב-"ב.מ.וו"
        bmw_ad_links = [
            f"{base_url}{link['href']}"
            for link in all_links
            if '/ad/' in link['href'] and link.get_text(strip=True).startswith('ב.מ.וו')
        ]
        
        # אם אין מודעות בדף הנוכחי, הפסיק את הלולאה
        if not bmw_ad_links:
            print(f"No more ads found. Stopping at page {page_index}.")
            break
        
        # לולאה לעבור על כל קישור למודעת ב.מ.וו
        for ad_url in bmw_ad_links:
            ad_response = requests.get(ad_url)
            
            if ad_response.status_code == 200:
                ad_soup = BeautifulSoup(ad_response.content, 'html.parser')
                
                # שליפת נתונים מהמודעה
                car_title = ad_soup.find('h2', class_='card-title').get_text(strip=True) if ad_soup.find('h2', class_='card-title') else 'N/A'
                description = ad_soup.find('div', class_='card-description').get_text(strip=True) if ad_soup.find('div', class_='card-description') else 'N/A'
                
                # פיצול הכותרת ליצרן ודגם
                manufacturer, model = car_title.split(' ', 1) if ' ' in car_title else (car_title, 'N/A')
                
                # שליפת פרטי רכב נוספים
                price_elements = ad_soup.find_all('h2', class_='card-title')
                price = 'N/A'
                if len(price_elements) > 1:
                    # בדיקה אם האלמנט השני הוא מחיר
                    price_candidate = price_elements[1].get_text(strip=True)
                    if '₪' in price_candidate or price_candidate.replace(',', '').isdigit():
                        price = price_candidate
                
                car_year = ad_soup.find('p', class_='card-text').get_text(strip=True) if ad_soup.find('p', class_='card-text') else 'N/A'
                
                # המרה לשנה ובדיקה אם בתוך הטווח
                try:
                    car_year = int(car_year)
                    if car_year < start_year or car_year > end_year:
                        continue  # דלג על מודעה אם השנה מחוץ לטווח
                except ValueError:
                    continue  # דלג על מודעה אם השנה אינה מספר תקין
                
                mileage = ad_soup.find('i', class_='fa fa-fw fa-tachometer-alt').find_next_sibling('span').get_text(strip=True) if ad_soup.find('i', class_='fa fa-fw fa-tachometer-alt') else 'N/A'
                hand = ad_soup.find('i', class_='fa fa-fw fa-hand-paper').find_next_sibling('span').get_text(strip=True) if ad_soup.find('i', class_='fa fa-fw fa-hand-paper') else 'N/A'
                fuel_type = ad_soup.find('i', class_='fa fa-fw fa-gas-pump').find_next_sibling('span').get_text(strip=True) if ad_soup.find('i', class_='fa fa-fw fa-gas-pump') else 'N/A'
                
                # שליפת פרטים נוספים ספציפיים
                description_element = ad_soup.find('p', class_='text-word-break')
                specific_description = description_element.get_text(strip=True) if description_element else 'N/A'
                
                date_elements = ad_soup.select('div.p-1 > div.d-flex.flex-row.align-items-center.justify-content-center.flex-wrap > div')
                creation_date = date_elements[0].get_text(strip=True).replace('תאריך יצירה: ', '') if len(date_elements) > 0 else 'N/A'
                last_bump_date = date_elements[1].get_text(strip=True).replace('תאריך הקפצה אחרון: ', '') if len(date_elements) > 1 else 'N/A'
                
                image_elements = ad_soup.select('div.col-12.d-flex.mt-3.justify-content-center.flex-wrap figure')
                num_images = len(image_elements)
                
                data = {
                    'שנה': car_year,
                    'יד': hand,
                    'ת. הילוכים': 'N/A',
                    'נפח': 'N/A',
                    'סוג מנוע': fuel_type,
                    'ק"מ': mileage,
                    'טסט עד': 'N/A',
                    'צבע': 'N/A',
                    'בעלות קודמת': 'N/A',
                    'בעלות נוכחית': 'N/A',
                    'אזור': 'N/A',
                    'עיר': 'N/A'
                }
                
                table = ad_soup.select_one('#tab-product .row table')
                if table:
                    for row in table.select('tr'):
                        cells = row.select('td')
                        if len(cells) == 2:
                            key = cells[0].get_text(strip=True)
                            value = cells[1].get_text(strip=True)
                            if key in data:
                                data[key] = value

                # שמירת הנתונים ברשימה
                data1.append([
                    manufacturer, car_year, model, data.get('יד'), data.get('ת. הילוכים'), data.get('נפח'), 
                    data.get('סוג מנוע'), data.get('בעלות קודמת'), data.get('בעלות נוכחית'), data.get('אזור'), data.get('עיר'), 
                    price, num_images, creation_date, last_bump_date, specific_description, data.get('צבע'), data.get('ק"מ'), data.get('טסט עד')
                ])
            else:
                print(f"Failed to retrieve ad page: {ad_url}")
    else:
        print(f"Failed to retrieve page: {page_url}")

    # הגדלת האינדקס של הדף
    page_index += 1

# יצירת DataFrame בעזרת pandas
df2 = pd.DataFrame(data1, columns=[
    'יצרן', 'שנה', 'דגם', 'יד', 'ת. הילוכים', 'נפח', 'סוג מנוע', 
    'בעלות קודמת', 'בעלות נוכחית', 'אזור', 'עיר', 'מחיר', 
    'מספר תמונות', 'תאריך יצירה', 'תאריך הקפצה אחרון', 'תיאור', 'צבע', 'ק"מ', 'טסט עד'
])


def days_until_test(test_date_str):
    try:
        # בדיקה אם הקלט ריק
        if not test_date_str:
            return None  # החזרת None עבור קלט ריק
        
        # המרת תאריך הטסט לתוך אובייקט datetime
        test_date = datetime.strptime(test_date_str, '%m/%Y')
        
        # קביעת התאריך ליום האחרון בחודש
        next_month = test_date.replace(day=28) + timedelta(days=4)  # מביא אותנו ליום כלשהו בחודש הבא
        test_date = next_month - timedelta(days=next_month.day)  # מביא אותנו ליום האחרון של החודש
        
        # מציאת היום הנוכחי
        today = datetime.now()
        
        # חישוב מספר הימים עד לטסט
        days_left = (test_date - today).days
        
        return days_left
    except Exception as e:
        print("An error occurred while calculating days until test:", e)
        return None  # החזרת None במקרה של שגיאהבמקרה של שגיאה


# החלת הפונקציה על העמודה 'טסט עד'
df2['טסט עד'] = df2['טסט עד'].apply(days_until_test)

No more ads found. Stopping at page 4.
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' does not match format '%m/%Y'
An error occurred while calculating days until test: time data 'N/A' d

In [6]:
# הגדרת מילון עם שמות העמודות החדשות
new_column_names = {
    'יצרן': 'manufactor',
    'שנה': 'Year',
    'דגם': 'model',
    'יד': 'Hand',
    'ת. הילוכים': 'Gear',
    'נפח': 'Engine_capacity',
    'סוג מנוע': 'Engine_type',
    'בעלות קודמת': 'Prev_ownership',
    'בעלות נוכחית': 'Curr_ownership',
    'אזור': 'Area',
    'עיר': 'City',
    'מחיר': 'Price',
    'מספר תמונות': 'Pic_num',
    'תאריך יצירה': 'Cre_date',
    'תאריך הקפצה אחרון': 'Repub_date',
    'תיאור': 'Description',
    'צבע': 'Color',
    'ק"מ': 'Km',
    'טסט עד': 'Test'
    
}

# שינוי שמות העמודות
df2.rename(columns=new_column_names, inplace=True)

In [7]:
df2['Test'] = df2['Test'].astype(pd.Int64Dtype())

לאחר הבנה מעמיקה יותר לגבי הטיפוס הקטגוריאלי בנינו ליסטים לכל הטיפוסים לפי סינון ומציאת ערכים בכל עמודה 
ומה האפשרויות הקימות באופן כללי כדי להכניס לרשימה שמתוכה יבחר כל פעם הקטגוריה

In [8]:
# הגדרת הערכים המותרים לעמודת Engine_type
lst = ['בנזין', 'דיזל', 'גז', 'היברידי', 'חשמלי']

# בדיקת ערכים לא חוקיים בעמודת Engine_type והחלפתם ב-None
for i in range(len(df2)):
    if df2.loc[i, 'Engine_type'] not in lst:
        df2.loc[i, 'Engine_type'] = None

# הגדרת העמודה כקטגוריה
df2['Engine_type'] = df2['Engine_type'].astype('category')

In [9]:
# הגדרת הערכים המותרים לעמודת Gear
gear_list = ['אוטומטית', 'ידנית', 'טיפטרוניק', 'רובוטית']

# בדיקת ערכים לא חוקיים בעמודת Gear והחלפתם ב-None
for i in range(len(df2)):
    if df2.loc[i, 'Gear'] not in gear_list:
        df2.loc[i, 'Gear'] = None

# הגדרת העמודה כקטגוריה
df2['Gear'] = df2['Gear'].astype('category')

In [10]:
# הגדרת הערכים המותרים לעמודת Curr_ownership
ownership_list = ['פרטית', 'חברה', 'השכרה', 'ליסינג', 'מונית', 'לימוד נהיגה', 'ייבוא אישי', 'ממשלתי', 'אחר']

# בדיקת ערכים לא חוקיים בעמודת Curr_ownership והחלפתם ב-None
for i in range(len(df2)):
    if df2.loc[i, 'Curr_ownership'] not in ownership_list:
        df2.loc[i, 'Curr_ownership'] = None

# הגדרת העמודה כקטגוריה
df2['Curr_ownership'] = df2['Curr_ownership'].astype('category')

In [11]:
# הגדרת הערכים המותרים לעמודת Prev_ownership
ownership_list = ['פרטית', 'חברה', 'השכרה', 'ליסינג', 'מונית', 'לימוד נהיגה', 'ייבוא אישי', 'ממשלתי', 'אחר']

# בדיקת ערכים לא חוקיים בעמודת Prev_ownership והחלפתם ב-None
for i in range(len(df2)):
    if df2.loc[i, 'Prev_ownership'] not in ownership_list:
        df2.loc[i, 'Prev_ownership'] = None

# הגדרת העמודה כקטגוריה
df2['Prev_ownership'] = df2['Prev_ownership'].astype('category')

החלפנו את הטיפוסים של העמודות בדאטה שלנו לפי הבקשה במטלה 

In [12]:
# קבלת הערכים הייחודיים בעמודת model
unique_models = df2['model'].unique()

# המרת עמודת model לעמודה קטגוריאלית עם הקטגוריות הייחודיות
df2['model'] = pd.Categorical(df2['model'], categories=unique_models)

In [13]:
df2['Price'] = pd.to_numeric(df2['Price'].str.replace(',', '').str.replace('₪', ''), errors='coerce')

In [14]:
# המרה של עמודות לערכים מספריים
numeric_columns = ['Engine_capacity', 'Km']
for col in numeric_columns:
    if col in df2.columns:
        # בדיקה אם העמודה היא מסוג מחרוזת לפני השימוש ב- str.replace
        if df2[col].dtype == object:
            df2[col] = pd.to_numeric(df2[col].str.replace(',', ''), errors='coerce')
        else:
            df2[col] = pd.to_numeric(df2[col], errors='coerce')
    else:
        print(f"Warning: Column '{col}' not found in DataFrame")

# המרה של עמודות לתאריכים
date_columns = ['Cre_date', 'Repub_date']
for col in date_columns:
    if col in df2.columns:
        df2[col] = pd.to_datetime(df2[col], errors='coerce')
    else:
        print(f"Warning: Column '{col}' not found in DataFrame")

# עמודות שצריכות להישאר כמחרוזות
string_columns = ['Area', 'City', 'Description', 'Color']
for col in string_columns:
    if col in df2.columns:
        df2[col] = df2[col].astype(str)
    else:
        print(f"Warning: Column '{col}' not found in DataFrame")

# המרה של עמודות לערכים שלמים (int)
integer_columns = ['Year', 'Hand', 'Engine_capacity', 'Pic_num', 'Km']
for col in integer_columns:
    if col in df2.columns:
        df2[col] = pd.to_numeric(df2[col], errors='coerce').fillna(0).astype(int)
    else:
        print(f"Warning: Column '{col}' not found in DataFrame")

# הצגת סוגי הנתונים של DataFrame לאחר השינוי
print(df2.dtypes)


manufactor                 object
Year                        int32
model                    category
Hand                        int32
Gear                     category
Engine_capacity             int32
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                     int32
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                      object
Km                          int32
Test                        Int64
dtype: object


לפי פיתון הטיפוס object הוא בעצם Str

In [15]:
df2

,manufactor,Year,model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,ב.מ.וו,1997,Z3,5,ידנית,1900,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,79000.0,3,2024-06-13,2024-06-13,****למכירה***\n Z3 \nמודל - 1997 ידני \nיד -04...,כסף מטלי,226000,78
1,ב.מ.וו,2022,סדרה 1,1,אוטומטית,1500,בנזין,פרטית,פרטית,תל אביב,תל אביב יפו,138500.0,7,2024-08-06,2024-08-06,"ב מ וו, סדרה 1, בצבע כחול מטאלי, שנת 2022, 350...",כחול כהה מטאלי,35000,231
2,ב.מ.וו,2016,סדרה 3,2,אוטומטית,2000,בנזין,פרטית,פרטית,באר שבע והסביבה,באר שבע,80000.0,9,2024-01-06,2024-05-31,ב.מ.וו סידרה 3 i320\nGT\n2000 טווין טורבו \nיד...,לבן,122000,78
3,ב.מ.וו,2008,סדרה 5,3,אוטומטית,3000,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,60000.0,7,2024-04-30,2024-04-30,יש הרבה תוספות,לבן,220000,320
4,ב.מ.וו,2016,סדרה 4,6,אוטומטית,3000,בנזין,פרטית,פרטית,N/A,ג'וליס,170000.0,4,2024-03-04,2024-04-28,435i...מנוע 3000 טוין טורבו ...ללא תאונות יפה ...,שחור,90000,139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,ב.מ.וו,2021,330,2,אוטומטית,2000,בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,305000.0,0,2022-07-20,2022-07-20,רכב חדש יפיפה &lt;br/&gt;&lt;br/&gt;פירוט:&lt;...,N/A,20000,<NA>
115,ב.מ.וו,2007,320,2,טיפטרוניק,1997,בנזין,NaN,NaN,זכרון - בנימינה,זכרון יעקב,26000.0,0,2022-01-07,2022-01-07,רכב מטופל במוסך ב מ וו מצב מכני מעולה &lt;br/&...,N/A,313000,<NA>
116,ב.מ.וו,2017,סדרה 1,2,אוטומטית,2000,בנזין,NaN,NaN,N/A,נתנייה,90000.0,0,2022-11-06,2022-11-06,רכב חדש לגמרי ללא תאונות 2017 יד שניה . פרטית ...,N/A,79000,<NA>
117,ב.מ.וו,2018,סדרה 5,3,אוטומטית,1998,היברידי,NaN,NaN,תל אביב,תל אביב יפו,199000.0,0,2022-04-25,2022-04-25,אוטו נדיר מתחייב בבדיקה \r\nכל השיפורים נראה מ...,N/A,99000,<NA>


קריאה לקובץ CSV במחשב שלי

In [16]:
df2.to_csv(r'C:\Users\USER\Desktop\car_data.csv', index=False, encoding='utf-8-sig')

שאלת רשות

ייבאנו דאטה חדש ממשרד התחבורה הוספנו עמודה בשם מודל שתואמת לעמודה בדאטה המקורי שלנו על ידי המילון שיצרנו שינינו את הכותרת של השנים לכותרת שתואמת לעמודה בדאטה המקורי שלנו ולקחנו את מספר הרכבים הפעילים לפי הכוונה של המרצה.

In [17]:
import requests
import pandas as pd
import re

# URL של ה-API
base_api_url = "https://data.gov.il/api/3/action/datastore_search"
resource_id = "5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6"
limit = 5000

# פונקציה לקביעת הסדרה על בסיס דגם הרכב
# בניית פונקציה לקביעת הסדרה על בסיס דגם הרכב
def determine_series(model):
    series_patterns = {
        'סדרה 1': (r'116I|118I|120I|1\d{2}[iI]', 'סדרה 1'),
        'סדרה 2': (r'2\d{2}[iI]', 'סדרה 2'),
        'סדרה 3': (r'3\d{2}[iI]|318|320|316|330|325|328', 'סדרה 3'),
        'סדרה 4': (r'A 4-I 613|4\d{2}[iI]', 'סדרה 4'),
        'סדרה 5': (r'5\d{2}[iI]|528|530|540|523|525', 'סדרה 5'),
        'סדרה 6': (r'645CI|6\d{2}[iI]', 'סדרה 6'),
        'סדרה 7': (r'730D|7\d{2}[iI]|740|750', 'סדרה 7'),
        'סדרה 8': (r'8\d{2}[iI]', 'סדרה 8'),
        'X1': (r'X1\b', 'X1'),
        'X2': (r'X2\b', 'X2'),
        'X3': (r'X3\b', 'X3'),
        'X4': (r'X4\b', 'X4'),
        'X5': (r'X5\b', 'X5'),
        'X6': (r'X6\b', 'X6'),
        'X7': (r'X7\b', 'X7'),
        'Z3': (r'Z3\b|במוו 8\.1 3Z\b', 'Z3'),
        'Z4': (r'Z4\b', 'Z4'),
        'M1': (r'M1\b', 'M1'),
        'M2': (r'M2\b', 'M2'),
        'M3': (r'M3\b', 'M3'),
        'M4': (r'M4\b', 'M4'),
        'M5': (r'M5\b', 'M5'),
        'M6': (r'M6\b', 'M6'),
        'M8': (r'M8\b', 'M8'),
        'סדרה 3 קופה/ קבריולט': (r'סדרה 3 קופה/ קבריולט', 'סדרה 3 קופה/ קבריולט'),
    }
    for series, (pattern, key) in series_patterns.items():
        if re.search(pattern, model, re.IGNORECASE):
            return key
    return 'אחר'  # אם לא נמצא תבנית מתאימה

# לולאה לריצה על כל הדפים
offset = 0
all_records = []

while True:
    # בקשת הנתונים מה-API
    response = requests.get(f"{base_api_url}?resource_id={resource_id}&limit={limit}&offset={offset}")
    data = response.json()
    records = data['result']['records']
    
    # בדיקה אם יש נתונים חדשים
    if not records:
        break

    all_records.extend(records)
    offset += limit

# המרת הנתונים ל-DataFrame של Pandas
df = pd.DataFrame(all_records)

# סינון רכבים מסוג "ב מ וו"
df_bmw = df[df['tozeret_nm'].str.contains('ב מ וו', case=False, na=False)]

# הוספת עמודת model על בסיס הפונקציה
df_bmw['model'] = df_bmw['kinuy_mishari'].apply(determine_series)

# ביצוע סכימה של העמודה mispar_rechavim_pailim לפי העמודות shnat_yitzur ו-model
result_df = df_bmw.groupby(['shnat_yitzur', 'model'])['mispar_rechavim_pailim'].sum().reset_index()

# שינוי שם העמודה shnat_yitzur ל-Year ושם העמודה mispar_rechavim_pailim ל-Supply_score
result_df = result_df.rename(columns={'shnat_yitzur': 'Year', 'mispar_rechavim_pailim': 'Supply_score'})

result_df

<ipython-input-17-256e29091b9c>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bmw['model'] = df_bmw['kinuy_mishari'].apply(determine_series)


,Year,model,Supply_score
0,1996,אחר,19
1,1996,סדרה 3,5
2,1996,סדרה 4,1
3,1996,סדרה 5,0
4,1997,Z3,11
...,...,...,...
364,2024,סדרה 3,292
365,2024,סדרה 4,125
366,2024,סדרה 5,53
367,2024,סדרה 7,25


In [18]:
# יצירת מילון שממפה דגמים לסדרות
model_to_series = {
    'סדרה 1': ['120i', 'סדרה 1'],
    'סדרה 2': ['סדרה 2'],
    'סדרה 3': ['318', '320', '316', '330', '325', '328', 'סדרה 3 קופה/ קבריולט', 'סדרה 3'],
    'סדרה 4': ['A 4-I 613', 'סדרה 4'],
    'סדרה 5': ['528', '530', '540', '525', '523', 'סדרה 5'],
    'סדרה 6': ['645CI', 'סדרה 6'],
    'סדרה 7': ['750', '740', 'סדרה 7'],
    'סדרה 8': ['סדרה 8'],
    'X1': ['X1'],
    'X2': ['X2'],
    'X3': ['X3'],
    'X4': ['X4'],
    'X5': ['X5'],
    'X6': ['X6'],
    'X7': ['X7'],
    'Z3': ['Z3'],
    'Z4': ['Z4'],
    'M1': ['M1'],
    'M2': ['M2'],
    'M3': ['M3'],
    'M4': ['M4'],
    'M5': ['M5'],
    'M6': ['M6'],
    'M8': ['M8'],
    # ניתן להוסיף סדרות נוספות לפי הצורך
}

# הוספת עמודת סדרות לדאטה df2 על בסיס המילון
def map_model_to_series(model):
    for series, models in model_to_series.items():
        if model in models:
            return series
    return 'אחר'

df2['series'] = df2['model'].apply(map_model_to_series)

# מיזוג הדאטה df2 עם result_df על בסיס העמודות Year ו-series
df2_temp = df2.copy()
df2_temp['series'] = df2_temp['model'].apply(map_model_to_series)

# מיזוג הנתונים
df2_merged = df2_temp.merge(result_df, left_on=['Year', 'series'], right_on=['Year', 'model'], how='left')

# מילוי ערכים חסרים ב-0
df2_merged['Supply_score'].fillna(0, inplace=True)

# שינוי עמודת Supply_score ל-INT
df2_merged['Supply_score'] = df2_merged['Supply_score'].astype(int)

# הסרת עמודת series לאחר המיזוג אם אינה נדרשת
df2_merged.drop(columns=['series', 'model_y'], inplace=True)
df2_merged.rename(columns={'model_x': 'model'}, inplace=True)

# הצגת הדאטה המצטבר
print(df2_merged.head())


  manufactor  Year   model  Hand      Gear  Engine_capacity Engine_type  \
0     ב.מ.וו  1997      Z3     5     ידנית             1900       בנזין   
1     ב.מ.וו  2022  סדרה 1     1  אוטומטית             1500       בנזין   
2     ב.מ.וו  2016  סדרה 3     2  אוטומטית             2000       בנזין   
3     ב.מ.וו  2008  סדרה 5     3  אוטומטית             3000       בנזין   
4     ב.מ.וו  2016  סדרה 4     6  אוטומטית             3000       בנזין   

  Prev_ownership Curr_ownership             Area         City     Price  \
0          פרטית          פרטית  ירושלים והסביבה      ירושלים   79000.0   
1          פרטית          פרטית          תל אביב  תל אביב יפו  138500.0   
2          פרטית          פרטית  באר שבע והסביבה      באר שבע   80000.0   
3          פרטית          פרטית  ירושלים והסביבה      ירושלים   60000.0   
4          פרטית          פרטית              N/A       ג'וליס  170000.0   

   Pic_num   Cre_date Repub_date  \
0        3 2024-06-13 2024-06-13   
1        7 2024-08-06 2024

In [19]:
df2_merged

,manufactor,Year,model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,ב.מ.וו,1997,Z3,5,ידנית,1900,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,79000.0,3,2024-06-13,2024-06-13,****למכירה***\n Z3 \nמודל - 1997 ידני \nיד -04...,כסף מטלי,226000,78,11
1,ב.מ.וו,2022,סדרה 1,1,אוטומטית,1500,בנזין,פרטית,פרטית,תל אביב,תל אביב יפו,138500.0,7,2024-08-06,2024-08-06,"ב מ וו, סדרה 1, בצבע כחול מטאלי, שנת 2022, 350...",כחול כהה מטאלי,35000,231,263
2,ב.מ.וו,2016,סדרה 3,2,אוטומטית,2000,בנזין,פרטית,פרטית,באר שבע והסביבה,באר שבע,80000.0,9,2024-01-06,2024-05-31,ב.מ.וו סידרה 3 i320\nGT\n2000 טווין טורבו \nיד...,לבן,122000,78,488
3,ב.מ.וו,2008,סדרה 5,3,אוטומטית,3000,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,60000.0,7,2024-04-30,2024-04-30,יש הרבה תוספות,לבן,220000,320,213
4,ב.מ.וו,2016,סדרה 4,6,אוטומטית,3000,בנזין,פרטית,פרטית,N/A,ג'וליס,170000.0,4,2024-03-04,2024-04-28,435i...מנוע 3000 טוין טורבו ...ללא תאונות יפה ...,שחור,90000,139,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,ב.מ.וו,2021,330,2,אוטומטית,2000,בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,305000.0,0,2022-07-20,2022-07-20,רכב חדש יפיפה &lt;br/&gt;&lt;br/&gt;פירוט:&lt;...,N/A,20000,<NA>,833
115,ב.מ.וו,2007,320,2,טיפטרוניק,1997,בנזין,NaN,NaN,זכרון - בנימינה,זכרון יעקב,26000.0,0,2022-01-07,2022-01-07,רכב מטופל במוסך ב מ וו מצב מכני מעולה &lt;br/&...,N/A,313000,<NA>,139
116,ב.מ.וו,2017,סדרה 1,2,אוטומטית,2000,בנזין,NaN,NaN,N/A,נתנייה,90000.0,0,2022-11-06,2022-11-06,רכב חדש לגמרי ללא תאונות 2017 יד שניה . פרטית ...,N/A,79000,<NA>,526
117,ב.מ.וו,2018,סדרה 5,3,אוטומטית,1998,היברידי,NaN,NaN,תל אביב,תל אביב יפו,199000.0,0,2022-04-25,2022-04-25,אוטו נדיר מתחייב בבדיקה \r\nכל השיפורים נראה מ...,N/A,99000,<NA>,464


קריאה לקובץ CSV של הרשות במחשב שלי

In [20]:
df2_merged.to_csv(r'C:\Users\USER\Desktop\car_all_data_reshot.csv', index=False, encoding='utf-8-sig')